In [67]:
import pandas as pd
import os
import csv
import time
import sys
from collections import Counter
import re

pd.options.mode.chained_assignment = None


In [6]:
csv.field_size_limit(sys.maxsize)
syntgram_dir = "../data"
filename = "find_with.tsv"

raw_data = []

with open(syntgram_dir + "/" + filename) as f:
    reader = csv.reader(f, delimiter='\t', quotechar = None, doublequote= True)
    raw_data = [r for r in reader]
    
df = pd.DataFrame(data=raw_data)

df = df[[0,1,2]]
df.columns = ["verb", "arg_structure", "count"]

print(df)


         verb                                      arg_structure count
0        find  '/CC/cc/2 find/VB/ROOT/0 sent/VBD/parataxis/2 ...    11
1        find  '/CC/cc/3 but/CC/cc/3 find/VB/ROOT/0 within/IN...    11
2        find  '/CC/cc/3 you/PRP/nsubj/3 find/VB/ROOT/0 withi...    11
3        find  '/CC/dep/2 find/VB/dep/0 's/VBZ/ccomp/2 withou...    11
4        find  '/IN/prep/3 may/NNP/pobj/1 find/VBP/ccomp/0 fa...    10
...       ...                                                ...   ...
173664  finds  by/IN/prep/3 reason/NN/pobj/1 finds/VBZ/ROOT/0...    16
173665  finds  by/IN/prep/3 search/NN/pobj/1 finds/VBZ/ccomp/...    14
173666  finds  by/IN/prep/3 time/NN/pobj/1 finds/VBZ/ROOT/0 s...    11
173667  finds  by/IN/prep/3 vaughan/NNP/pobj/1 saves/VBZ/advc...    34
173668  finds  by/IN/prep/4 del/NNP/nn/3 mar/NNP/pobj/1 finds...    14

[173669 rows x 3 columns]


In [7]:
sub_df = df[["arg_structure"]]
print(sub_df)

                                            arg_structure
0       '/CC/cc/2 find/VB/ROOT/0 sent/VBD/parataxis/2 ...
1       '/CC/cc/3 but/CC/cc/3 find/VB/ROOT/0 within/IN...
2       '/CC/cc/3 you/PRP/nsubj/3 find/VB/ROOT/0 withi...
3       '/CC/dep/2 find/VB/dep/0 's/VBZ/ccomp/2 withou...
4       '/IN/prep/3 may/NNP/pobj/1 find/VBP/ccomp/0 fa...
...                                                   ...
173664  by/IN/prep/3 reason/NN/pobj/1 finds/VBZ/ROOT/0...
173665  by/IN/prep/3 search/NN/pobj/1 finds/VBZ/ccomp/...
173666  by/IN/prep/3 time/NN/pobj/1 finds/VBZ/ROOT/0 s...
173667  by/IN/prep/3 vaughan/NNP/pobj/1 saves/VBZ/advc...
173668  by/IN/prep/4 del/NNP/nn/3 mar/NNP/pobj/1 finds...

[173669 rows x 1 columns]


In [17]:
sample = sub_df.iloc[896:898,].values
print(sample)

[['again/RB/advmod/2 find/VB/ccomp/0 favor/NN/dobj/2 with/IN/prep/2 writers/NNS/pobj/4']
 ['again/RB/advmod/2 find/VB/ccomp/0 graced/VBD/ccomp/2 with/IN/prep/3 retinue/NN/pobj/4']]


In [20]:
instrument = sample[0][0]
print(instrument)

again/RB/advmod/2 find/VB/ccomp/0 favor/NN/dobj/2 with/IN/prep/2 writers/NNS/pobj/4


In [ ]:
# df_subset["prepc_vmod"] = df_subset.arg_structure.str.contains("(?=prepc)(?=.*vmod)", regex=True)

In [43]:
def check_bias(root_verb, connection_word, arg_structure):
    print(arg_structure)
    holder = []
    root_index = -1
    connection = -1

    counter = 1
    for word in arg_structure.split(" "):
        temp = word.split("/")
        # print(temp)
        if temp[0] == connection_word:
            connection = int(temp[3])
        elif temp[0] == root_verb:
            root_index = counter
        else:
            counter = counter + 1
        holder.append(temp)

    print(root_verb, root_index)
    print(connection_word, connection)
    if root_index == connection:
        print("Instrument bias")
        return "instrument"
    else:
        print("Modifier bias")
        return "modifier"

In [62]:
def check_bias_alt(arg_structure):
    print(arg_structure)
    if "/prep/" not in arg_structure:
        return "neither"
    holder = []
    root_index = -1
    connection = -1

    counter = 1
    for word in arg_structure.split(" "):
        temp = word.split("/")
        if temp[2] == "prep":
            connection_word = temp[0]
            connection = int(temp[3])

        if temp[3] == "0":
            root_index = counter
            root_verb = temp[0]
        else:
            counter = counter + 1
        holder.append(temp)

    # print(root_verb, root_index)
    # print(connection_word, connection)
    if root_index == connection:
        # print("Instrument bias")
        return "instrument"
    else:
        # print("Modifier bias")
        return "modifier"

In [74]:
# check_bias_alt(instrument)

In [73]:
# check_bias_alt(sample[1][0])

In [ ]:
sub_df["bias"] = sub_df["arg_structure"].apply(check_bias_alt)

In [66]:
sub_df.to_csv("../data_output/test.csv")

In [72]:
a = "bat table car"
b = "car bat"

print(re.search("(?=bat)(?=.*car)", a) == None)
print(re.search("(?=bat)(?=.*car)", b) == None)


False
True
